In [1]:
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

In [2]:
f = open("/content/drive/MyDrive/Buffer Based CFP-Tree/DATASET/C500_SIZE/d10mil_9999804.txt", 'r')

lines = f.read().splitlines()
raw_true_data = {}
true_x = []
true_y = []
true_cluster = []
for line in lines:
  k = line.split(" ")
  true_x.append(float(k[0]))
  true_y.append(float(k[1]))
  true_cluster.append(int(k[2][1:]))

raw_true_data['X'] = true_x
raw_true_data['Y'] = true_y
raw_true_data['True Cluster'] = true_cluster

true_data = DataFrame(raw_true_data)

del raw_true_data

In [3]:
f = open("/content/drive/MyDrive/Buffer Based CFP-Tree/DATASET/1028/BIRCH/d10mil9999804_leaf_3171.txt", 'r')

lines = f.read().splitlines()
raw_data = {}
index_data = {}
x = []
y = []
index = []
for line in lines:
  k = line.split('|')
  coordinate = k[0].split(' ')
  x.append(float(coordinate[0]))
  y.append(float(coordinate[1]))

  tempIndex = []
  length = len(k[1])-1
  temp = k[1][1:length]
  leafIndex = temp.split(', ')
  for i in leafIndex:
    tempIndex.append(int(i))
  index.append(tempIndex)

raw_data['x'] = x
raw_data['y'] = y
index_data['x'] = x
index_data['y'] = y
index_data['Index'] = index

data = DataFrame(raw_data)
idata = DataFrame(index_data)
print(data)

del raw_data
del index_data

                x            y
0     8470.221120  6381.129543
1     8468.075150  6359.947783
2     8445.077217  6358.145125
3     8454.824619  6404.128565
4     8473.434739  6401.259837
...           ...          ...
3166  8679.401382  4110.972335
3167  8695.787184  4102.021591
3168  8686.535539  4123.456298
3169  8688.390382  4087.811075
3170  8658.299795  4121.365894

[3171 rows x 2 columns]


In [4]:
from sklearn.cluster import KMeans

#PARAMETER
Total_Number = 9999804
Total_Cluster = 500
Clustered_Line_Number = 3171

kmeans = KMeans(n_clusters = Total_Cluster)
kmeans.fit(data)

del data

idata['Predicted Cluster'] = kmeans.labels_
print(idata)

                x  ...  Predicted Cluster
0     8470.221120  ...                248
1     8468.075150  ...                248
2     8445.077217  ...                248
3     8454.824619  ...                248
4     8473.434739  ...                248
...           ...  ...                ...
3166  8679.401382  ...                150
3167  8695.787184  ...                150
3168  8686.535539  ...                150
3169  8688.390382  ...                150
3170  8658.299795  ...                150

[3171 rows x 4 columns]


In [5]:
#Purity Calculation

# P1 : ability of algorithm to put dissimlar points in different clusters
P1 = 0

# P2 : ability of algorithm to put similar points together in the same cluster
P2 = 0

clustered_data = {}
for i in range(Total_Cluster):
  clustered_data[i] = set()

for i in range(Clustered_Line_Number):
  for index in idata['Index'][i]:
    clustered_data[idata['Predicted Cluster'][i]].add(index)

del idata
#print(clustered_data)

In [6]:
real_data = {}

for i in range(Total_Cluster):
  real_data[i] = set()

for i in range(Total_Number):
  real_data[true_data['True Cluster'][i]].add(i)

#print(real_data)

In [7]:
# P1 Calculation
for j in range(Total_Cluster):
  max_intersect = 0
  for i in range(Total_Cluster):
    if len(clustered_data[j] & real_data[i]) > max_intersect:
      max_intersect = len(clustered_data[j] & real_data[i])

  P1 += max_intersect

In [8]:
# P2 Calculation
for i in range(Total_Cluster):
  max_intersect = 0
  for j in range(Total_Cluster):
    if len(clustered_data[j] & real_data[i]) > max_intersect:
      max_intersect = len(clustered_data[j] & real_data[i])

  P2 += max_intersect

In [9]:
P1_Percentage = (float(P1) / float(Total_Number))

print("P1 : ", P1_Percentage)

P2_Percentage = (float(P2) / float(Total_Number))

print("P2 : ", P2_Percentage)

P1 :  0.9574618662525786
P2 :  0.9840779879285634
